In [15]:
!pip install text2emotion

In [16]:
import pandas as pd
import re
import glob 

import nltk
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


import text2emotion as te

[nltk_data] Downloading package vader_lexicon to C:\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [36]:

def getJSONData(file):
    df = pd.read_json(file, lines=True)
    data = df[df.lang == "en"].loc[:, ['content']]
    data["sentiments"] = data.content.apply(lambda x: getSentimentEmotion(x))
    data[['neg','neu', 'pos','comp', 'angry', 'fear', 'happy', 'sad', 'surprise']] = data['sentiments'].str.split(',',expand=True)
    data.drop(["sentiments"], inplace=True, axis = 1)
    return data

In [37]:

def getSentimentEmotion(tweet):
    sid = SentimentIntensityAnalyzer()
    tweet_words = [t.lower() for t in tweet.split() if re.match(r"(\w+)", t)]
    text = " ".join(tweet_words)
    ss = sid.polarity_scores(text)
    emo = te.get_emotion(text)
    return f'{ss["neg"]}, {ss["neu"]}, {ss["pos"]}, {ss["compound"]}, {emo["Angry"]}, {emo["Fear"]}, {emo["Happy"]}, {emo["Sad"]}, {emo["Surprise"]}'

In [39]:
FILE_DIR = "Data/tweets/"
overalldf = pd.DataFrame()
csv_files = glob.glob(f"{FILE_DIR}*.json")
for file in csv_files: 
    print (file)
    data = getJSONData(file)
    data.to_csv(f"{file.split('.')[0]}.csv")
    overalldf = overalldf.append(data)


Data/tweets\13 reasons why.json
Data/tweets\all of us are dead.json
Data/tweets\bridgerton.json
Data/tweets\maid.json
Data/tweets\money heist.json
Data/tweets\red notice.json
Data/tweets\squid game.json
Data/tweets\stranger things.json
Data/tweets\the witcher.json
Data/tweets\you.json


In [40]:
overalldf.to_csv(f"{FILE_DIR}processed/final.csv")